In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fbprophet import Prophet

In [2]:
newdf = pd.read_csv('/tmp/bitmex_ohlcv_XBTUSD_60', header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume'], parse_dates=['timestamp'])

In [4]:
newdf.timestamp = newdf.timestamp.map(lambda x: datetime.utcfromtimestamp(round(float(x))))

In [5]:
from datetime import timedelta
def myresample(df):
    df['dvolume']=df.close*df.volume
    mydf = df.set_index('timestamp')
    td = timedelta(minutes=datetime.now().minute)
    mydf.index = mydf.index - td
    mydf=mydf.resample('1H', loffset=td).agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum', 'dvolume':'sum'})
    mydf['vwap']=mydf.dvolume/mydf.volume
    mydf.drop('dvolume', axis=1, inplace = True)
    mydf['volatility']=mydf.vwap.rolling(24).std()*np.sqrt(24)
    return mydf

In [10]:
def prophesy(df, lookahead=24, freq2='1H',include_history=True):
    prodf = df.reset_index()[['timestamp', 'vwap']]
    prodf.vwap = np.log(prodf.vwap)
    prodf.columns = ['ds', 'y']
    pp = Prophet(changepoint_prior_scale=0.001)
    pp.fit(prodf)
    future = pp.make_future_dataframe(periods=lookahead, freq=freq2, include_history=include_history)
    forecast = pp.predict(future)
    #display(prodf)
    #display(forecast)
    #return pd.concat([prodf, forecast], axis=1)
    #return forecast, prodf
    return forecast.tail(1).yhat.iloc[0]-prodf.tail(1).y.iloc[0]

In [26]:
from time import strftime, mktime, sleep


The below is the main signal generator. I had been using just the absolute value of the signal (the second field in the output, *pp*, but it is more appropriate to use volatility units - curvol is the daily volatility, and so the quantity *pp/curvol* gives the expected gain in units of vol. A value around $\pm 1$ should be the cutoff. (Negative value for short, positive for long).

In [ ]:
while True:
    newdf = pd.read_csv('/tmp/bitmex_ohlcv_XBTUSD_60', header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume'], parse_dates=['timestamp'])
    newdf.timestamp = newdf.timestamp.map(lambda x: datetime.utcfromtimestamp(round(float(x))))
    foo = myresample(newdf.tail(6000))
    curvol = foo.tail(1).iloc[0]['volatility']
    pp = prophesy(foo)
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), pp, curvol, pp/curvol, file=open('/tmp/predict.txt','a'))
    sleep(300)
                     

/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithC

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override

/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithC

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override

/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithC

...

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override

/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithC

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override

/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithC

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/igor/anaconda3/envs/proph/lib/python3.7/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override